In [1]:
pip install neo4j

Note: you may need to restart the kernel to use updated packages.


In [2]:
 from neo4j import GraphDatabase

In [3]:
class Neo4jConnection:
    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        if self.driver is not None:
            self.driver.close()

# Метод, который передает запрос в БД
    def query(self, query, db=None):
        assert self.driver is not None, "Driver not initialized!"
        session = None
        response = None
        try:
            session = self.driver.session(database=db) if db is not None else self.driver.session()
            response = list(session.run(query))
        except Exception as e:
            print("Query failed:", e)
        finally:
            if session is not None:
                session.close()
        return response

In [4]:
conn = Neo4jConnection(uri="bolt://localhost:7687", user="gnbaltaeva", password="kakdela2101")
conn.query("CREATE OR REPLACE DATABASE lab01")

[]

In [6]:
query_string = '''
// создаем узлы
CREATE (board:Board {name: "Доска"})
CREATE (chalk:Chalk {name: "Мел"})
CREATE (door:Door {name: "Дверь"})
FOREACH(number IN range(1, 3) |
  CREATE (window:Window {name: "Окно " + number})
)
FOREACH(number IN range(1, 10) |
  CREATE (table:Table {name: "Стол " + number})
)
FOREACH(number IN range(1, 10) |
  CREATE (chair:Chair {name: "Стул " + number})
)
FOREACH(number IN range(1, 10) |
  CREATE (computer:Computer {name: "Компьютер " + number})
)
'''
conn.query(query_string, db='lab01')

[]

In [9]:
query_string = '''
// стулья
UNWIND range(1, 10) AS chair_number
MATCH (chair:Chair {name: "Стул " + chair_number}), (table:Table {name: "Стол " + chair_number})
CREATE (chair)-[:находится_у]->(table)

'''
conn.query(query_string, db='lab01')

[]

In [12]:
query_string = '''
// компьютеры
UNWIND range(1, 10) AS number
MATCH (computer:Computer {name: "Компьютер " + number}), (table:Table {name: "Стол " + number})
CREATE (computer)-[:находится_на]->(table)

'''
conn.query(query_string, db='lab01')


[]

In [13]:
query_string = '''
// столы
MATCH (table1:Table {name: "Стол 1"})
MATCH (table2:Table {name: "Стол 2"})
MATCH (table3:Table {name: "Стол 3"})
MATCH (table4:Table {name: "Стол 4"})
MATCH (table5:Table {name: "Стол 5"})
MATCH (table6:Table {name: "Стол 6"})
MATCH (table7:Table {name: "Стол 7"})
MATCH (table8:Table {name: "Стол 8"})
MATCH (table9:Table {name: "Стол 9"})
MATCH (table10:Table {name: "Стол 10"})

CREATE (table1)-[:спереди]->(table2),
(table2)-[:сзади]->(table1),
(table2)-[:слева]->(table3),
(table2)-[:спереди]->(table5),
(table3)-[:спереди]->(table6),
(table3)-[:слева]->(table4),
(table3)-[:справа]->(table2),
(table4)-[:спереди]->(table7),
(table4)-[:справа]->(table3),
(table5)-[:сзади]->(table2),
(table5)-[:слева]->(table6),
(table5)-[:спереди]->(table8),
(table6)-[:сзади]->(table3),
(table6)-[:спереди]->(table9),
(table6)-[:слева]->(table7),
(table6)-[:справа]->(table5),
(table7)-[:сзади]->(table4),
(table7)-[:справа]->(table6),
(table7)-[:спереди]->(table10),
(table8)-[:сзади]->(table5),
(table8)-[:слева]->(table9),
(table9)-[:сзади]->(table6),
(table9)-[:слева]->(table10),
(table9)-[:справа]->(table8),
(table10)-[:справа]->(table9),
(table10)-[:сзади]->(table7)

'''
conn.query(query_string, db='lab01')

[]

In [14]:
query_string = '''
// Остальное
MATCH (стол1:Table {name: "Стол 1"})
MATCH (стол10:Table {name: "Стол 10"})
MATCH (мел:Chalk {name: "Мел"})
MATCH (дверь:Door {name: "Дверь"})
MATCH (доска:Board {name: "Доска"})
MATCH (окно1:Window {name: "Окно 1"})
MATCH (окно2:Window {name: "Окно 2"})
MATCH (окно3:Window {name: "Окно 3"})

CREATE (стол1)-[:рядом_с]->(окно1),
(окно1)-[:слева]->(окно2),
(окно2)-[:слева]->(окно3),
(окно3)-[:справа]->(окно2),
(окно2)-[:справа]->(окно1),
(доска)-[:рядом_с]->(окно3),
(мел)-[:лежит_на]->(доска),
(стол10)-[:рядом_с]->(дверь);


'''
conn.query(query_string, db='lab01')

[]